In [23]:
# 1. Dependencias e imports
!pip -q install altair pillow selenium altair_saver
!apt-get -qq update
!apt-get -qq install -y chromium-chromedriver

import os, pandas as pd, numpy as np
import altair as alt
from datetime import datetime
from pathlib import Path
from PIL import Image

# Configurar Chrome headless para exportar imágenes si usamos selenium
os.environ["CHROMIUM_FLAGS"] = "--headless"
os.environ["ALT_AIRCHROME_PATH"] = "/usr/bin/chromium-browser"
os.environ["WEBDRIVER_PATH"] = "/usr/bin/chromedriver"

print("✅ Dependencias listas.")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ Dependencias listas.


In [24]:
# 2. Utilidades
def normaliza_hoja(nombre: str) -> str:
    return nombre.strip().lower().replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u")

def long_from_excel(xlsx_path: Path, posibles_hojas=("Pistacho","Nuez","Almendra")) -> pd.DataFrame:
    xls = pd.ExcelFile(xlsx_path)
    mapping = {}
    for hoja in xls.sheet_names:
        norm = normaliza_hoja(hoja)
        mapping[norm] = hoja
    frames = []
    for etiqueta in posibles_hojas:
        key = normaliza_hoja(etiqueta)
        # Buscar coincidencia flexible
        coincidente = None
        for k,v in mapping.items():
            if key in k or k in key:
                coincidente = v
                break
        if coincidente is None:
            raise ValueError(f"No se encontró la hoja para '{etiqueta}'. Hojas disponibles: {xls.sheet_names}")
        df = pd.read_excel(xls, sheet_name=coincidente)
        # Detectar columnas: Google Trends suele exportar Fecha + Valor en primeras 2 columnas
        df = df.dropna(how='all')
        # Buscar primera columna datetime/fecha
        df.columns = [str(c) for c in df.columns]
        # Tomar primeras dos columnas
        df2 = df.iloc[:, :2].copy()
        df2.columns = ["Fecha", "Valor"]
        df2["Fecha"] = pd.to_datetime(df2["Fecha"], errors="coerce")
        df2 = df2.dropna(subset=["Fecha"])  # quitar encabezados extra
        # A veces Valor viene con texto tipo '<1'; lo convertimos seguro
        df2["Valor"] = pd.to_numeric(df2["Valor"].astype(str).str.replace("<", "", regex=False), errors="coerce")
        df2 = df2.dropna(subset=["Valor"])  # quitar vacíos
        df2["Producto"] = etiqueta.capitalize()
        frames.append(df2)
    base = pd.concat(frames, ignore_index=True)
    base["Año"] = base["Fecha"].dt.year
    base["Mes"] = base["Fecha"].dt.month
    # Agregación mensual (promedio de las semanas)
    base["AñoMes"] = base["Fecha"].dt.to_period('M')
    base_m = (base.groupby(["Producto","AñoMes"], as_index=False)
                   ["Valor"].mean().round(0))
    base_m["Año"] = base_m["AñoMes"].dt.year
    base_m["Mes"] = base_m["AñoMes"].dt.month
    base_m["Fecha"] = pd.to_datetime(base_m["Año"].astype(str) + '-' + base_m["Mes"].astype(str) + '-15')
    base_m = base_m[["Producto","Año","Mes","Valor","Fecha"]]
    return base_m.sort_values(["Producto","Fecha"]).reset_index(drop=True)

def valida_extremos(df: pd.DataFrame):
    tiene_0 = (df['Valor'] == 0).any()
    tiene_100 = (df['Valor'] == 100).any()
    print(f"¿Existe 0?: {tiene_0} | ¿Existe 100?: {tiene_100}")
    return tiene_0, tiene_100

print("✅ Utilidades cargadas.")


✅ Utilidades cargadas.


In [25]:
# 3. Carga de datos (elige una opción)
from pathlib import Path
root = Path('.')

# csv_path = Path('../base_frutossecos_2018_2025.csv')  # opción A: CSV ya limpio
# xlsx_path = Path('../Base de datos inicial.xlsx')     # opción B: Excel multitimeline
csv_path = Path('/content/base_antigua.csv') # Use the available CSV file

if csv_path.exists():
    print("🔹 Cargando CSV limpio existente...")
    df = pd.read_csv(csv_path, skiprows=1) # Skip the first row
    # Handle the specific structure of base_antigua.csv
    df = df.reset_index()
    df = df.melt(id_vars=['Semana'], var_name='Producto', value_name='Valor')
    df = df.rename(columns={'Semana': 'Fecha'})
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
    df = df.dropna(subset=['Fecha']) # Drop rows where date conversion failed

    # Extract Year and Month after converting to datetime
    df['Año'] = df['Fecha'].dt.year
    df['Mes'] = df['Fecha'].dt.month

    # Clean up 'Producto' column names
    df['Producto'] = df['Producto'].str.split(':').str[0].str.strip()
    # Filter out rows where Producto is 'index'
    df = df[df['Producto'] != 'index'].copy()

    # Aggregate to monthly level
    df["AñoMes"] = df["Fecha"].dt.to_period('M')
    df = (df.groupby(["Producto","AñoMes"], as_index=False)
                   ["Valor"].mean().round(0))
    df["Año"] = df["AñoMes"].dt.year
    df["Mes"] = df["AñoMes"].dt.month
    df["Fecha"] = pd.to_datetime(df["Año"].astype(str) + '-' + df["Mes"].astype(str) + '-15')
    df = df[["Producto","Año","Mes","Valor","Fecha"]]
    df = df.sort_values(["Producto","Fecha"]).reset_index(drop=True)


# elif xlsx_path.exists():
#     print("🔹 Convirtiendo Excel multitimeline a formato largo...")
#     df = long_from_excel(xlsx_path)
#     # Guardar CSV limpio para futuras ejecuciones
#     out_csv = Path('../base_frutossecos_2018_2025.csv')
#     df.to_csv(out_csv, index=False)
#     print(f"✅ CSV guardado en: {out_csv}")
else:
    raise FileNotFoundError(f"No se encontró el archivo en: {csv_path}. Asegúrate de que el archivo está presente.")

print(df.head())
print(df.dtypes)

🔹 Cargando CSV limpio existente...
          Producto   Año  Mes  Valor      Fecha
0  chocolate dubai  2020    9    0.0 2020-09-15
1  chocolate dubai  2020   10    0.0 2020-10-15
2  chocolate dubai  2020   11    0.0 2020-11-15
3  chocolate dubai  2020   12    0.0 2020-12-15
4  chocolate dubai  2021    1    0.0 2021-01-15
Producto            object
Año                  int64
Mes                  int64
Valor              float64
Fecha       datetime64[ns]
dtype: object


In [26]:
# 4. Validaciones rápidas
print(f"Filas totales: {len(df):,}")
dups = df.duplicated(subset=["Producto","Año","Mes"]).sum()
print(f"Duplicados (Producto, Año, Mes): {dups}")
t0, t100 = valida_extremos(df)

assert df['Valor'].between(0,100).all(), "Hay valores fuera de 0–100, revisa la base."
assert dups == 0, "Hay duplicados por mes y producto; revisa la agregación."
print("✅ Validaciones OK")


Filas totales: 183
Duplicados (Producto, Año, Mes): 0
¿Existe 0?: True | ¿Existe 100?: False
✅ Validaciones OK


In [27]:
# 5. Visualización con Altair
alt.data_transformers.disable_max_rows()

chart = (
    alt.Chart(df)
    .mark_line(point=True)
    .encode(
        x=alt.X('Fecha:T', title='Año'),
        y=alt.Y('Valor:Q', title='Interés relativo (0–100)'),
        color=alt.Color('Producto:N', title='Fruto seco'),
        tooltip=['Producto','Año','Mes','Valor']
    )
    .properties(title='Interés por frutos secos en Chile (2018–2025)', width=800, height=400)
)
chart


alt.Chart(...)

In [22]:
# 6. Exportación: HTML y JPG
out_dir = Path('../visualizacion')
out_dir.mkdir(parents=True, exist_ok=True)
html_path = out_dir / 'vis_pistacho.html'
png_path = out_dir / 'vis_pistacho.png'
jpg_path = out_dir / 'vis_pistacho.jpg'

# HTML (nativo)
chart.save(str(html_path))

# Imagen: intentar con altair_saver; si falla, captura con Selenium
saved = False
try:
    from altair_saver import save as alt_save
    alt_save(chart, str(png_path))
    saved = True
    print("✅ PNG exportado con altair_saver")
except Exception as e:
    print("⚠️ altair_saver falló, usando Selenium como respaldo...", e)
    # Respaldo Selenium (captura del HTML)
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from time import sleep
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1200,800")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get('file://' + str(html_path.resolve()))
    sleep(2)
    driver.save_screenshot(str(png_path))
    driver.quit()
    saved = True
    print("✅ PNG exportado con Selenium")

if saved:
    # Convertir a JPG
    im = Image.open(png_path).convert('RGB')
    im.save(jpg_path, 'JPEG', quality=95)
    print(f"✅ Exportados: {html_path.name}, {jpg_path.name}")
else:
    print("❌ No fue posible exportar imagen. Revisa dependencias.")

sorted(os.listdir(out_dir))


⚠️ altair_saver falló, usando Selenium como respaldo... No matches for version='5.20.1' among ['4.0.2', '4.8.1', '4.17.0'].
Often this can be fixed by updating altair_viewer:
    pip install -U altair_viewer
✅ PNG exportado con Selenium
✅ Exportados: vis_pistacho.html, vis_pistacho.jpg


['vis_pistacho.html', 'vis_pistacho.jpg', 'vis_pistacho.png']